In [1]:
## scraping the data from the website "octopios.com"
from bs4 import BeautifulSoup
import requests

In [2]:
url= "https://octopios.com/"

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")






In [3]:
print(soup.prettify())  # This will print the HTML content in a readable format

<!DOCTYPE html>
<html lang="en">
 <head>
  <div wire:effects="[]" wire:id="SagPRjPFqIchHqNG66qs" wire:snapshot='{"data":[],"memo":{"id":"SagPRjPFqIchHqNG66qs","name":"trader::layouts.header","path":"\/","method":"GET","children":[],"scripts":[],"assets":[],"errors":[],"locale":"ar"},"checksum":"8d729f606c30480c42597ef0d0fde1ece58e98bf56bfd664e749e7613c164d88"}'>
   <meta charset="utf-8"/>
   <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
   <link href="https://octopios.com/uploads/logo.png" rel="shortcut icon" type="image"/>
   <link href="https://octopios.com/uploads/logo.png" rel="shortcut icon" type="images/x-icon"/>
   <link href="https://octopios.com/uploads/logo.png" rel="icon" type="image/png"/>
   <meta content="PureSoft" name="author"/>
   <!-- Favicon -->
   <link as="font" crossorigin="anonymous" href="https://octopios.com/site/fonts/octopios.woff?png09e" rel="preload" type="font/woff"/>
   <!-- Plugins CSS -->
   <link href="https

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_products():
    url = "https://octopios.com/trader/bestselling"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract product information
        products = []
        # You'll need to inspect the HTML to find correct selectors
        
        return products
    except Exception as e:
        print(f"Error scraping: {e}")
        return []

In [6]:
print(soup.prettify())  # This will print the HTML content in a readable format

<!DOCTYPE html>
<html lang="en">
 <head>
  <div wire:effects="[]" wire:id="SagPRjPFqIchHqNG66qs" wire:snapshot='{"data":[],"memo":{"id":"SagPRjPFqIchHqNG66qs","name":"trader::layouts.header","path":"\/","method":"GET","children":[],"scripts":[],"assets":[],"errors":[],"locale":"ar"},"checksum":"8d729f606c30480c42597ef0d0fde1ece58e98bf56bfd664e749e7613c164d88"}'>
   <meta charset="utf-8"/>
   <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
   <link href="https://octopios.com/uploads/logo.png" rel="shortcut icon" type="image"/>
   <link href="https://octopios.com/uploads/logo.png" rel="shortcut icon" type="images/x-icon"/>
   <link href="https://octopios.com/uploads/logo.png" rel="icon" type="image/png"/>
   <meta content="PureSoft" name="author"/>
   <!-- Favicon -->
   <link as="font" crossorigin="anonymous" href="https://octopios.com/site/fonts/octopios.woff?png09e" rel="preload" type="font/woff"/>
   <!-- Plugins CSS -->
   <link href="https

In [7]:
from fake_useragent import UserAgent

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import os
from fake_useragent import UserAgent
from urllib.parse import urljoin, urlparse
import re
from typing import List, Dict

class OctopiosWebScraper:
    def __init__(self):
        # Initialize user agent for rotating headers
        self.ua = UserAgent()
        self.base_url = "https://octopios.com/"
        self.session = requests.Session()
        
        # Set up headers to avoid being blocked
        self.headers = {
            'User-Agent': self.ua.random,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
        }
        self.session.headers.update(self.headers)
        
        # Storage for scraped data
        self.products = []
        self.categories = []
        self.all_scraped_data = {}
    
    def get_page_content(self, url, delay=2):
        """Safely fetch page content with error handling"""
        try:
            print(f"Scraping: {url}")
            time.sleep(delay)  # Be respectful to the server
            
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            
            return BeautifulSoup(response.content, 'html.parser')
        
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return None
    
    def extract_product_info(self, product_element):
        """Extract product information from HTML element"""
        product_data = {
            'name': '',
            'price': '',
            'description': '',
            'image_url': '',
            'product_url': '',
            'category': '',
            'availability': '',
            'rating': '',
            'features': []
        }
        
        try:
            # Extract product name
            name_elem = product_element.find(['h1', 'h2', 'h3', 'h4'], class_=re.compile(r'product|title|name', re.I))
            if not name_elem:
                name_elem = product_element.find(['a', 'span'], class_=re.compile(r'product|title|name', re.I))
            if name_elem:
                product_data['name'] = name_elem.get_text(strip=True)
            
            # Extract price
            price_elem = product_element.find(['span', 'div', 'p'], class_=re.compile(r'price|cost|amount', re.I))
            if not price_elem:
                price_elem = product_element.find(string=re.compile(r'\$|€|£|\d+\.\d+'))
                if price_elem:
                    price_elem = price_elem.parent
            if price_elem:
                product_data['price'] = price_elem.get_text(strip=True)
            
            # Extract description
            desc_elem = product_element.find(['p', 'div'], class_=re.compile(r'description|summary|details', re.I))
            if desc_elem:
                product_data['description'] = desc_elem.get_text(strip=True)
            
            # Extract image URL
            img_elem = product_element.find('img')
            if img_elem and img_elem.get('src'):
                img_url = img_elem.get('src')
                product_data['image_url'] = urljoin(self.base_url, img_url)
            
            # Extract product URL
            link_elem = product_element.find('a', href=True)
            if link_elem:
                product_data['product_url'] = urljoin(self.base_url, link_elem['href'])
            
            # Extract any additional features or specifications
            feature_elems = product_element.find_all(['li', 'span'], class_=re.compile(r'feature|spec|attribute', re.I))
            for feature in feature_elems:
                feature_text = feature.get_text(strip=True)
                if feature_text and len(feature_text) > 3:
                    product_data['features'].append(feature_text)
        
        except Exception as e:
            print(f"Error extracting product data: {e}")
        
        return product_data
    
    def scrape_bestselling_page(self):
        """Scrape the bestselling products page"""
        url = "https://octopios.com/"
        soup = self.get_page_content(url)
        
        if not soup:
            print("Failed to load bestselling page")
            return []
        
        products = []
        
        # Try different common product container selectors
        product_selectors = [
            '.product-item',
            '.product-card',
            '.product',
            '[class*="product"]',
            '.item',
            '.card',
            '[data-product]'
        ]
        
        product_elements = []
        for selector in product_selectors:
            elements = soup.select(selector)
            if elements:
                product_elements = elements
                print(f"Found {len(elements)} products using selector: {selector}")
                break
        
        # If no specific selectors work, try to find products by common patterns
        if not product_elements:
            # Look for divs that might contain product info
            all_divs = soup.find_all('div')
            product_elements = [div for div in all_divs if 
                             div.find('img') and 
                             (div.find(string=re.compile(r'\$|\d+\.\d+')) or 
                              div.find(['span', 'div'], class_=re.compile(r'price', re.I)))]
        
        print(f"Processing {len(product_elements)} potential product elements...")
        
        for element in product_elements:
            product_data = self.extract_product_info(element)
            if product_data['name']:  # Only add if we found a name
                products.append(product_data)
        
        return products
    
    def scrape_additional_pages(self):
        """Scrape other important pages for more context"""
        pages_to_scrape = [
            '/trader/bestselling',
            '/trader/mostpopular',
            '/trader/categories/1',
            '/trader/categories/2',
            '/trader/categories/3',
            '/trader/categories/4',
            '/trader/categories/5',
            '/trader/categories/6',
            '/trader/categories/7',
            '/trader/categories/8'
        ]
        
        additional_data = {}
        
        for page_path in pages_to_scrape:
            url = urljoin(self.base_url, page_path)
            soup = self.get_page_content(url)
            
            if soup:
                # Extract text content for general knowledge
                text_content = soup.get_text(separator=' ', strip=True)
                additional_data[page_path] = {
                    'title': soup.title.string if soup.title else '',
                    'content': text_content[:2000],  # Limit content length
                    'url': url
                }
        
        return additional_data
    
    def extract_navigation_links(self, soup):
        """Extract navigation and category links"""
        nav_links = []
        
        # Find navigation menus
        nav_elements = soup.find_all(['nav', 'ul'], class_=re.compile(r'nav|menu', re.I))
        
        for nav in nav_elements:
            links = nav.find_all('a', href=True)
            for link in links:
                link_text = link.get_text(strip=True)
                link_url = urljoin(self.base_url, link['href'])
                
                if link_text and len(link_text) > 1:
                    nav_links.append({
                        'text': link_text,
                        'url': link_url,
                        'type': 'navigation'
                    })
        
        return nav_links
    
    def save_data(self, filename_prefix='octopios_data'):
        """Save all scraped data in multiple formats"""
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        
        # Save products as CSV
        if self.products:
            df = pd.DataFrame(self.products)
            csv_filename = f"{filename_prefix}_products_{timestamp}.csv"
            df.to_csv(csv_filename, index=False)
            print(f"Products saved to: {csv_filename}")
        
        # Save all data as JSON
        json_filename = f"{filename_prefix}_complete_{timestamp}.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(self.all_scraped_data, f, indent=2, ensure_ascii=False)
        print(f"Complete data saved to: {json_filename}")
        
        return csv_filename, json_filename
    
    def run_complete_scrape(self):
        """Execute the complete scraping process"""
        print("🚀 Starting Octopios.com scraping...")
        print("=" * 50)
        
        # Step 1: Scrape bestselling products
        print("📦 Scraping bestselling products...")
        self.products = self.scrape_bestselling_page()
        print(f"Found {len(self.products)} products")
        
        # Step 2: Get main page content
        print("🏠 Scraping main page...")
        main_soup = self.get_page_content(self.base_url)
        if main_soup:
            nav_links = self.extract_navigation_links(main_soup)
            print(f"Found {len(nav_links)} navigation links")
        else:
            nav_links = []
        
        # Step 3: Scrape additional pages
        print("📄 Scraping additional pages...")
        additional_data = self.scrape_additional_pages()
        
        # Step 4: Compile all data
        self.all_scraped_data = {
            'products': self.products,
            'navigation_links': nav_links,
            'additional_pages': additional_data,
            'scrape_timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
            'total_products': len(self.products)
        }
        
        # Step 5: Save data
        print("💾 Saving scraped data...")
        csv_file, json_file = self.save_data()
        
        print("=" * 50)
        print("✅ Scraping completed successfully!")
        print(f"📊 Total products found: {len(self.products)}")
        print(f"📁 Files created: {csv_file}, {json_file}")
        
        return self.all_scraped_data

def main():
    """Main function to run the scraper"""
    # Create scraper instance
    scraper = OctopiosWebScraper()
    
    # Run the complete scraping process
    data = scraper.run_complete_scrape()
    
    # Display sample results
    print("\n📋 Sample Product Data:")
    print("-" * 30)
    for i, product in enumerate(data['products'][:3], 1):
        print(f"{i}. {product['name']}")
        print(f"   Price: {product['price']}")
        print(f"   Description: {product['description'][:100]}...")
        print()
    
    print("🎉 Scraping complete! You can now use this data for your chatbot.")
    return data

if __name__ == "__main__":
    # Run the scraper
    scraped_data = main()
    
    # Optional: Print summary statistics
    print(f"\n📈 Summary:")
    print(f"   Products with names: {len([p for p in scraped_data['products'] if p['name']])}")
    print(f"   Products with prices: {len([p for p in scraped_data['products'] if p['price']])}")
    print(f"   Products with descriptions: {len([p for p in scraped_data['products'] if p['description']])}")

🚀 Starting Octopios.com scraping...
📦 Scraping bestselling products...
Scraping: https://octopios.com/
Found 19 products using selector: .product
Processing 19 potential product elements...
Found 19 products
🏠 Scraping main page...
Scraping: https://octopios.com/
Found 84 navigation links
📄 Scraping additional pages...
Scraping: https://octopios.com/trader/bestselling
Scraping: https://octopios.com/trader/mostpopular
Scraping: https://octopios.com/trader/categories/1
Scraping: https://octopios.com/trader/categories/2
Scraping: https://octopios.com/trader/categories/3
Scraping: https://octopios.com/trader/categories/4
Scraping: https://octopios.com/trader/categories/5
Scraping: https://octopios.com/trader/categories/6
Scraping: https://octopios.com/trader/categories/7
Scraping: https://octopios.com/trader/categories/8
💾 Saving scraped data...
Products saved to: octopios_data_products_20250819_192341.csv
Complete data saved to: octopios_data_complete_20250819_192341.json
✅ Scraping compl

In [ ]:
def scrape_bestselling_page(self):
        """Scrape the bestselling products page"""
        url = f"https://octopios.com/"
        soup = self.get_page_content(url)
        
        if not soup:
            print("Failed to load bestselling page")
            return []
        
        products = []
        
        # Try different common product container selectors
        product_selectors = [
            '.product-item',
            '.product-card',
            '.product',
            '[class*="product"]',
            '.item',
            '.card',
            '[data-product]'
        ]
        
        product_elements = []
        for selector in product_selectors:
            elements = soup.select(selector)
            if elements:
                product_elements = elements
                print(f"Found {len(elements)} products using selector: {selector}")
                break
        
        # If no specific selectors work, try to find products by common patterns
        if not product_elements:
            # Look for divs that might contain product info
            all_divs = soup.find_all('div')
            product_elements = [div for div in all_divs if 
                             div.find('img') and 
                             (div.find(string=re.compile(r'\$|\d+\.\d+')) or 
                              div.find(['span', 'div'], class_=re.compile(r'price', re.I)))]
        
        print(f"Processing {len(product_elements)} potential product elements...")
        
        for element in product_elements:
            product_data = self.extract_product_info(element)
            if product_data['name']:  # Only add if we found a name
                products.append(product_data)
        
        return products

In [21]:
import json
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import urljoin, urlparse
import logging

class JSONLinkScraper:
    def __init__(self, json_file_path, delay=1):
        """
        Initialize the scraper with JSON file path and delay between requests
        
        Args:
            json_file_path (str): Path to your JSON file
            delay (int): Delay in seconds between requests to be respectful
        """
        self.json_file_path = json_file_path
        self.delay = delay
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        
        # Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    def load_json_data(self):
        """Load and parse the JSON file"""
        try:
            with open(self.json_file_path, 'r', encoding='utf-8') as file:
                return json.load(file)
        except Exception as e:
            self.logger.error(f"Error loading JSON file: {e}")
            return None
    
    def extract_links(self, data):
        """Extract all unique links from the JSON data"""
        links = set()
        
        # Extract from navigation_links
        if 'navigation_links' in data:
            for link in data['navigation_links']:
                if 'url' in link and link['url']:
                    links.add(link['url'])
        
        # Extract from additional_pages
        if 'additional_pages' in data:
            for url in data['additional_pages'].keys():
                # Convert relative URLs to absolute URLs
                if url.startswith('/'):
                    base_url = "https://octopios.com"  # Based on your data
                    full_url = urljoin(base_url, url)
                    links.add(full_url)
                else:
                    links.add(url)
        
        # Extract product URLs if they exist
        if 'products' in data:
            for product in data['products']:
                if 'product_url' in product and product['product_url']:
                    links.add(product['product_url'])
        
        return list(links)
    
    def scrape_page(self, url):
        """Scrape meaningful data from a single page"""
        try:
            self.logger.info(f"Scraping: {url}")
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract meaningful data
            page_data = {
                'url': url,
                'title': self.extract_title(soup),
                'meta_description': self.extract_meta_description(soup),
                'headings': self.extract_headings(soup),
                'products': self.extract_products(soup),
                'contact_info': self.extract_contact_info(soup),
                'prices': self.extract_prices(soup),
                'images': self.extract_images(soup),
                'text_content': self.extract_text_content(soup),
                'status_code': response.status_code,
                'page_size': len(response.content)
            }
            
            return page_data
            
        except requests.RequestException as e:
            self.logger.error(f"Error scraping {url}: {e}")
            return {
                'url': url,
                'error': str(e),
                'status_code': None
            }
    
    def extract_title(self, soup):
        """Extract page title"""
        title_tag = soup.find('title')
        return title_tag.get_text().strip() if title_tag else None
    
    def extract_meta_description(self, soup):
        """Extract meta description"""
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        return meta_desc.get('content', '').strip() if meta_desc else None
    
    def extract_headings(self, soup):
        """Extract all headings (h1-h6)"""
        headings = {}
        for i in range(1, 7):
            heading_tags = soup.find_all(f'h{i}')
            if heading_tags:
                headings[f'h{i}'] = [tag.get_text().strip() for tag in heading_tags]
        return headings
    
    def extract_products(self, soup):
        """Extract product information"""
        products = []
        
        # Common product selectors (adjust based on website structure)
        product_selectors = [
            '.product',
            '.product-item',
            '.product-card',
            '[data-product]',
            '.item'
        ]
        
        for selector in product_selectors:
            product_elements = soup.select(selector)
            if product_elements:
                for element in product_elements:
                    product = {
                        'name': self.extract_product_name(element),
                        'price': self.extract_product_price(element),
                        'image': self.extract_product_image(element),
                        'description': self.extract_product_description(element)
                    }
                    if any(product.values()):  # Only add if we found some data
                        products.append(product)
                break  # Use the first selector that finds products
        
        return products
    
    def extract_product_name(self, element):
        """Extract product name from element"""
        name_selectors = ['.product-name', '.product-title', 'h3', 'h4', '.title']
        for selector in name_selectors:
            name_element = element.select_one(selector)
            if name_element:
                return name_element.get_text().strip()
        return None
    
    def extract_product_price(self, element):
        """Extract product price from element"""
        price_selectors = ['.price', '.product-price', '.cost', '[data-price]']
        for selector in price_selectors:
            price_element = element.select_one(selector)
            if price_element:
                return price_element.get_text().strip()
        return None
    
    def extract_product_image(self, element):
        """Extract product image from element"""
        img_element = element.find('img')
        if img_element:
            return img_element.get('src') or img_element.get('data-src')
        return None
    
    def extract_product_description(self, element):
        """Extract product description from element"""
        desc_selectors = ['.description', '.product-desc', '.summary']
        for selector in desc_selectors:
            desc_element = element.select_one(selector)
            if desc_element:
                return desc_element.get_text().strip()
        return None
    
    def extract_contact_info(self, soup):
        """Extract contact information"""
        contact_info = {}
        
        # Phone numbers
        phone_pattern = r'[\+]?[1-9]?[0-9]{7,15}'
        phones = soup.find_all(text=lambda text: text and any(char.isdigit() for char in text))
        contact_info['phones'] = [phone.strip() for phone in phones if len(phone.strip()) >= 7][:5]  # Limit to 5
        
        # Email addresses
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        emails = soup.find_all(text=lambda text: text and '@' in text)
        contact_info['emails'] = [email.strip() for email in emails if '@' in email][:5]  # Limit to 5
        
        return contact_info
    
    def extract_prices(self, soup):
        """Extract all prices from the page"""
        price_patterns = [
            r'\$[\d,]+\.?\d*',
            r'[\d,]+\.?\d*\s*EGP',
            r'[\d,]+\.?\d*\s*USD',
            r'[\d,]+\.?\d*\s*EUR'
        ]
        
        prices = []
        page_text = soup.get_text()
        
        import re
        for pattern in price_patterns:
            found_prices = re.findall(pattern, page_text)
            prices.extend(found_prices)
        
        return list(set(prices))[:10]  # Limit to 10 unique prices
    
    def extract_images(self, soup):
        """Extract image URLs"""
        images = []
        img_tags = soup.find_all('img')
        
        for img in img_tags[:10]:  # Limit to 10 images
            src = img.get('src') or img.get('data-src')
            if src:
                images.append({
                    'src': src,
                    'alt': img.get('alt', ''),
                    'title': img.get('title', '')
                })
        
        return images
    
    def extract_text_content(self, soup):
        """Extract clean text content (first 500 characters)"""
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        
        text = soup.get_text()
        # Clean up text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text[:500] + "..." if len(text) > 500 else text
    
    def scrape_all_links(self, max_links=None):
        """Scrape all links from the JSON file"""
        data = self.load_json_data()
        if not data:
            return []
        
        links = self.extract_links(data)
        if max_links:
            links = links[:max_links]
        
        self.logger.info(f"Found {len(links)} unique links to scrape")
        
        scraped_data = []
        
        for i, url in enumerate(links, 1):
            self.logger.info(f"Progress: {i}/{len(links)}")
            
            page_data = self.scrape_page(url)
            scraped_data.append(page_data)
            
            # Be respectful - add delay between requests
            if i < len(links):
                time.sleep(self.delay)
        
        return scraped_data
    
    def save_to_csv(self, scraped_data, output_file='scraped_data.csv'):
        """Save scraped data to CSV file"""
        if not scraped_data:
            self.logger.warning("No data to save")
            return
        
        # Flatten the data for CSV
        flattened_data = []
        for page in scraped_data:
            flat_page = {
                'url': page.get('url'),
                'title': page.get('title'),
                'meta_description': page.get('meta_description'),
                'h1_headings': '; '.join(page.get('headings', {}).get('h1', [])),
                'h2_headings': '; '.join(page.get('headings', {}).get('h2', [])),
                'product_count': len(page.get('products', [])),
                'price_count': len(page.get('prices', [])),
                'image_count': len(page.get('images', [])),
                'text_preview': page.get('text_content', '')[:200],
                'status_code': page.get('status_code'),
                'page_size': page.get('page_size'),
                'error': page.get('error')
            }
            flattened_data.append(flat_page)
        
        df = pd.DataFrame(flattened_data)
        df.to_csv(output_file, index=False, encoding='utf-8')
        self.logger.info(f"Data saved to {output_file}")
    
    def save_to_json(self, scraped_data, output_file='scraped_data.json'):
        """Save scraped data to JSON file"""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(scraped_data, f, indent=2, ensure_ascii=False)
        self.logger.info(f"Data saved to {output_file}")


# Example usage
def main():
    # Initialize the scraper
    scraper = JSONLinkScraper('octopios_data_complete_20250819_190012.json', delay=2)  # 2 second delay between requests
    
    # Scrape all links (or limit with max_links parameter)
    scraped_data = scraper.scrape_all_links(max_links=10)  # Limit to 10 for testing
    
    # Save results
    scraper.save_to_csv(scraped_data, 'octopios_scraped_data.csv')
    scraper.save_to_json(scraped_data, 'octopios_scraped_data.json')
    
    # Print summary
    print(f"\nScraping completed!")
    print(f"Total pages scraped: {len(scraped_data)}")
    print(f"Successful scrapes: {len([p for p in scraped_data if 'error' not in p])}")
    print(f"Failed scrapes: {len([p for p in scraped_data if 'error' in p])}")

if __name__ == "__main__":
    main()

INFO:__main__:Found 10 unique links to scrape
INFO:__main__:Progress: 1/10
INFO:__main__:Scraping: https://octopios.com/vendors/packages
C:\Users\hp\AppData\Local\Temp\ipykernel_7880\1097244674.py:189: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phones = soup.find_all(text=lambda text: text and any(char.isdigit() for char in text))
C:\Users\hp\AppData\Local\Temp\ipykernel_7880\1097244674.py:194: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  emails = soup.find_all(text=lambda text: text and '@' in text)
INFO:__main__:Progress: 2/10
INFO:__main__:Scraping: https://octopios.com/trader/freelancer
INFO:__main__:Progress: 3/10
INFO:__main__:Scraping: https://octopios.com/trader/categories/7
INFO:__main__:Progress: 4/10
INFO:__main__:Scraping: https://octopios.com/trader/packages
INFO:__main__:Progress: 5/10
INFO:__main__:Scraping: https://octopios.com/vendors/login
INFO:__main__:


Scraping completed!
Total pages scraped: 10
Successful scrapes: 10
Failed scrapes: 0
